In [ ]:
import torch
import numpy as np
import pandas as pd

from model import PrefixNet

In [ ]:
from sklearn.manifold import TSNE, Isomap, MDS
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

cfg= {'batch_size':8,'prefix_len':5,'embed_size_per_token':300,'speaker_size':106,'epoch':12,'encoded':'encoded_st_p_ids','freeze_gpt':False}
name_col= 'st_p' # st_p, parl_part, encoded_bioguide_ids

model = PrefixNet(cfg)

model_name=f'24may_prefix_tuning_stp_prlen{cfg["prefix_len"]}_embsize{cfg["embed_size_per_token"]}_speaksize{cfg["speaker_size"]}_maxseqlen256_batch8_8_epoch{cfg["epoch"]}.pt'
checkpoint = torch.load('/cluster/scratch/goezsoy/nlp_lss_checkpoints/'+model_name,map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

X = model.embedding_layer.weight.detach().numpy()


X_embedded_pca = PCA(n_components=2).fit_transform(X)
X_embedded_tsne = TSNE(n_components=2).fit_transform(X)
X_embedded_mds = MDS(n_components=2).fit_transform(X)
X_embedded_isomap = Isomap(n_components=2).fit_transform(X)

speaker_labels = np.arange(0,cfg['speaker_size'])
processed_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df.csv')
meta_df= processed_df[processed_df[cfg['encoded']].isin(speaker_labels)].drop_duplicates(subset=cfg['encoded'])

#first plot
group = meta_df.sort_values(by=cfg['encoded'],ascending=True, inplace=False).term_party.values
cdict = {'Democrat': 'blue', 'Republican': 'red', 'Independent':'yellow'}

fig, ax = plt.subplots(figsize=(12,12))

for g in np.unique(group):
    ix = np.where(group == g)
    ax.scatter(X_embedded_tsne[ix,0], X_embedded_tsne[ix,1], c = cdict[g], label = g, s = 100)

for i, idx in enumerate(list(speaker_labels)):
    ax.annotate(meta_df[meta_df[cfg['encoded']]==idx][name_col].values[0], (X_embedded_tsne[i,0], X_embedded_tsne[i,1]))
ax.legend()
ax.set_title('tsne')

plt.show()

#second plot
fig2, ax2 = plt.subplots(figsize=(12,12))

for g in np.unique(group):
    ix = np.where(group == g)
    ax2.scatter(X_embedded_pca[ix,0], X_embedded_pca[ix,1], c = cdict[g], label = g, s = 100)

for i, idx in enumerate(list(speaker_labels)):
    ax2.annotate(meta_df[meta_df[cfg['encoded']]==idx][name_col].values[0], (X_embedded_pca[i,0], X_embedded_pca[i,1]))
ax2.legend()
ax2.set_title('pca')

plt.show()


#third plot
fig3, ax3 = plt.subplots(figsize=(12,12))

for g in np.unique(group):
    ix = np.where(group == g)
    ax3.scatter(X_embedded_mds[ix,0], X_embedded_mds[ix,1], c = cdict[g], label = g, s = 100)

for i, idx in enumerate(list(speaker_labels)):
    ax3.annotate(meta_df[meta_df[cfg['encoded']]==idx][name_col].values[0], (X_embedded_mds[i,0], X_embedded_mds[i,1]))
    
ax3.legend()
ax3.set_title('mds')

plt.show()

#fourth plot
fig4, ax4 = plt.subplots(figsize=(12,12))

for g in np.unique(group):
    ix = np.where(group == g)
    ax4.scatter(X_embedded_isomap[ix,0], X_embedded_isomap[ix,1], c = cdict[g], label = g, s = 100)

for i, idx in enumerate(list(speaker_labels)):
    ax4.annotate(meta_df[meta_df[cfg['encoded']]==idx][name_col].values[0], (X_embedded_isomap[i,0], X_embedded_isomap[i,1]))
    
ax4.legend()
ax4.set_title('isomap')

plt.show()

In [ ]:
# SAVE

# add perplexity score for each real sentence
import pandas as pd
import re

max_word_count = 256

valid_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/toy_processed_df_valid.csv')
temp_valid_df = valid_df.iloc[:100]


real_fake_df = pd.DataFrame(columns=['speech','label','perplexity'])

real_fake_df['speech'] = temp_valid_df['speech'].map(lambda row: ' '.join(row.split()[:max_word_count]))
real_fake_df['label'] = 1

generated_texts_path = '/cluster/home/goezsoy/K2T/results/50_keywordsets_eval/finetunedgptmed_valid100_trick/Result_w_5.0_nBeams_1_nGenSent_256_nWordsPerSent_1_topP_0.9_WC_glove_maxSENTENCES.txt'

file = open(generated_texts_path, 'r')

temp_speech = None
temp_perplexity = None

flag_speech = False
flag_perplexity = False

for line in file:
    if line != '\n' and re.search("\ASuccess_rate:", line) is None and re.search("#.:", line) is None:
        if re.search("\APerplexity:", line) is not None:
            temp_perplexity = line.split()[-1]
            flag_perplexity = True
        else:
            # remove <|endoftext|> tokens generated by k2t
            temp_speech = line.replace('<|endoftext|>','')

            # if initial text is <|endoftext|> removing it leads to
            # extra space at the start of sentence, so remove it
            if temp_speech[0] == ' ':
                temp_speech = temp_speech[1:]
            flag_speech = True
    
    if flag_speech and flag_perplexity:
        temp_df = pd.DataFrame.from_dict({'speech':[temp_speech],'label':[0],'perplexity':[temp_perplexity]})
        real_fake_df = pd.concat([real_fake_df,temp_df], ignore_index=True)

        flag_speech = False
        flag_perplexity = False

file.close()
real_fake_df.to_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/real_fake_df.csv', index=False)

In [ ]:
import glob
import os
results_path = '/cluster/home/goezsoy/conditioned_speech_gen/results/shard4'
folder_name = 'finetunedgptmed_valid10k'
experiment_name = 'Result_w_5.0_nBeams_1_nGenSent_256_nWordsPerSent_1_topP_0.9_WC_Guar_True_glove_maxSENTENCES.txt'

generated_texts_path = os.path.join(results_path,folder_name,experiment_name)

os.path.exists(generated_texts_path)

In [ ]:
# add perplexity score for each real sentence
import pandas as pd
import re
import os
import glob


max_word_count = 256

valid_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df_valid.csv')
temp_valid_df = valid_df.iloc[:500]


real_fake_df = pd.DataFrame(columns=['speech','label','perplexity'])

real_fake_df['speech'] = temp_valid_df['speech'].map(lambda row: ' '.join(row.split()[:max_word_count]))
real_fake_df['label'] = 1


results_path = '/cluster/home/goezsoy/conditioned_speech_gen/results'
folder_name = 'finetunedgptmed'
experiment_name = 'Result_w_5.0_nBeams_1_nGenSent_90_nWordsPerSent_1_topP_0.9_WC_Guar_True_glove_maxSENTENCES.txt'
shard_list = glob.glob(os.path.join(results_path,'shard*',folder_name))

for temp_shard in shard_list:
    
    generated_texts_path = os.path.join(temp_shard,experiment_name)

    file = open(generated_texts_path, 'r')

    temp_speech = None
    temp_perplexity = None

    flag_speech = False
    flag_perplexity = False

    for line in file:
        if line != '\n' and re.search("\ASuccess_rate:", line) is None and re.search("#.:", line) is None:
            if re.search("\APerplexity:", line) is not None:
                temp_perplexity = line.split()[-1]
                flag_perplexity = True
            else:
                # remove <|endoftext|> tokens generated by k2t
                temp_speech = line.replace('<|endoftext|>','')

                # if initial text is <|endoftext|> removing it leads to
                # extra space at the start of sentence, so remove it
                if temp_speech[0] == ' ':
                    temp_speech = temp_speech[1:]
                flag_speech = True
        
        if flag_speech and flag_perplexity:
            temp_df = pd.DataFrame.from_dict({'speech':[temp_speech],'label':[0],'perplexity':[temp_perplexity]})
            real_fake_df = pd.concat([real_fake_df,temp_df], ignore_index=True)

            flag_speech = False
            flag_perplexity = False

    file.close()
    
real_fake_df.to_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/real_fake_df.csv', index=False)

In [ ]:
real_fake_df.iloc[-6:].values

In [ ]:
valid_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df_valid.csv')

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-cased")

In [ ]:
def tokenize_function(row):

    tokenizer_dict = tokenizer(row['speech'])
    tokenizer_dict['labels'] = row['label']

    return {**tokenizer_dict}


def prepare_dataloader(df):

    dataset = Dataset.from_pandas(df)

    dataset = dataset.map(tokenize_function, batched=True)

    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    # for dynamic padding
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors='pt')

    dataloader = torch.utils.data.DataLoader(dataset,collate_fn=data_collator, batch_size=32, shuffle=True, drop_last=False)

    return dataloader

In [ ]:
real_fake_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/real_fake_df.csv')

real_fake_df['speech'] = real_fake_df['speech'].map(lambda row: row.lower())

# train vs test split
X_train_valid, X_test, _, _ = train_test_split(real_fake_df.index, real_fake_df['label'], test_size=0.2, random_state=0, stratify=real_fake_df['label'])

real_fake_train_valid_df = real_fake_df.iloc[X_train_valid].reset_index(drop=True)
real_fake_test_df = real_fake_df.iloc[X_test].reset_index(drop=True)

# train vs valid split
X_train, X_valid, _, _ = train_test_split(real_fake_train_valid_df.index, real_fake_train_valid_df['label'], test_size=0.1, random_state=0, stratify= real_fake_train_valid_df['label'])

real_fake_train_df = real_fake_train_valid_df.iloc[X_train].reset_index(drop=True)
real_fake_valid_df = real_fake_train_valid_df.iloc[X_valid].reset_index(drop=True)


train_dataloader = prepare_dataloader(real_fake_train_df)
valid_dataloader = prepare_dataloader(real_fake_valid_df)
test_dataloader = prepare_dataloader(real_fake_test_df)

In [ ]:
valid_dataloader

In [ ]:
for b in valid_dataloader:
    print(b)
    break

In [ ]:
ground_truth = b['labels']

logits = model(**b).logits
preds = torch.argmax(logits,axis=1)

In [ ]:
real_probs = torch.nn.functional.softmax(logits.cpu().detach(), dim=1)[:,1]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score


print(accuracy_score(ground_truth.numpy(), preds.numpy()))

print(roc_auc_score(ground_truth.numpy(), real_probs.numpy()))

print(average_precision_score(ground_truth.numpy(), real_probs.numpy()))

In [ ]:
import re
import os
import glob
import pandas as pd
import torch
import torch.optim as optim
import numpy as np
import random
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, average_precision_score, roc_auc_score

max_word_count = 100

# put real samples
real_fake_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/real_df_first5k.csv')

# put fake samples
results_path = '/cluster/home/goezsoy/conditioned_speech_gen/results'
folder_name = 'finetunedgptmed_lr2e5_epoch2'
experiment_name = 'Result_w_5.0_nBeams_1_nGenSent_128_nWordsPerSent_1_topP_0.9_WC_Guar_True_glove_maxSENTENCES.txt'

print(max_word_count,folder_name,experiment_name)
shard_list = glob.glob(os.path.join(results_path,'shard*',folder_name))


for temp_shard in shard_list:
    
    generated_texts_path = os.path.join(temp_shard,experiment_name)

    if os.path.exists(generated_texts_path):
        file = open(generated_texts_path, 'r')

        temp_speech = None
        temp_perplexity = None

        flag_speech = False
        flag_perplexity = False

        for line in file:
            if line != '\n' and re.search("\ASuccess_rate:", line) is None and re.search("#.:", line) is None:
                if re.search("\APerplexity:", line) is not None:
                    temp_perplexity = line.split()[-1]
                    flag_perplexity = True
                else:
                    # remove <|endoftext|> tokens generated by k2t
                    temp_speech = line.replace('<|endoftext|>','')

                    # if initial text is <|endoftext|> removing it leads to
                    # extra space at the start of sentence, so remove it
                    if temp_speech[0] == ' ':
                        temp_speech = temp_speech[1:]
                    flag_speech = True
            
            if flag_speech and flag_perplexity:
                temp_speech = ' '.join(temp_speech.split()[:max_word_count])
                temp_df = pd.DataFrame.from_dict({'speech':[temp_speech],'label':[0],'perplexity':[temp_perplexity]})
                real_fake_df = pd.concat([real_fake_df,temp_df], ignore_index=True)

                flag_speech = False
                flag_perplexity = False

        file.close()
    
real_fake_df['perplexity'] = real_fake_df['perplexity'].astype('float64')
real_fake_df.to_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/real_fake_df.csv', index=False)

In [ ]:
real_fake_test_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/real_fake_test_df.csv')

In [ ]:
real_fake_test_df[(real_fake_test_df['label']==1) & (real_fake_test_df['prediction']==0)]

In [3]:
import pandas as pd

train_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df_train.csv')

valid_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df_valid.csv')

In [35]:
text = 'Mr. President|| human, prisoners, president'

In [36]:
# || is used for separating prompt and keywords
intext_keyword_list = list(line.strip().split('|| '))
intext = intext_keyword_list[0]
keyword_list = list(intext_keyword_list[1].split(", "))
intext,keyword_list

('Mr. President', ['human', 'prisoners', 'president'])

In [34]:
line = 'Mr. President, human, prisoners, president'
intext_keyword_list = list(line.strip().split(", "))
intext_keyword_list[0],intext_keyword_list[1:]

('Mr. President', ['human', 'prisoners', 'president'])